In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame, functions as F

def get_spark(app_name: str):
    return (
        SparkSession.builder
        .appName(app_name)
        .master("local[*]")
        .config("spark.sql.session.timeZone", "UTC")
        .getOrCreate()
    )

spark = get_spark("example-app")



#### TASK 1

In [ ]:
df = spark.read.csv("C:\\Users\\g.aloisio\\Desktop\\weather-spark-app\\data\\raw\\weather_description.csv", header=True, inferSchema=True)

df = (
        df
        .withColumn("timestamp", F.to_timestamp("datetime"))
    )
city_columns = [c for c in df.columns if c not in ("datetime", "timestamp")]

long_df = df.select(
        "timestamp",
        F.expr(
            f"stack({len(city_columns)}, "
            + ", ".join([f"'{c}', `{c}`" for c in city_columns])
            + ") as (city, weather_description)"
        )
    )

long_df.filter(F.col("weather_description").isNotNull()).show()

+-------------------+-------------+-------------------+
|          timestamp|         city|weather_description|
+-------------------+-------------+-------------------+
|2012-10-01 12:00:00|        Eilat|               haze|
|2012-10-01 13:00:00|    Vancouver|               mist|
|2012-10-01 13:00:00|     Portland|   scattered clouds|
|2012-10-01 13:00:00|San Francisco|         light rain|
|2012-10-01 13:00:00|      Seattle|       sky is clear|
|2012-10-01 13:00:00|  Los Angeles|               mist|
|2012-10-01 13:00:00|    San Diego|       sky is clear|
|2012-10-01 13:00:00|    Las Vegas|       sky is clear|
|2012-10-01 13:00:00|      Phoenix|       sky is clear|
|2012-10-01 13:00:00|  Albuquerque|       sky is clear|
|2012-10-01 13:00:00|       Denver|         light rain|
|2012-10-01 13:00:00|  San Antonio|       sky is clear|
|2012-10-01 13:00:00|       Dallas|               mist|
|2012-10-01 13:00:00|      Houston|       sky is clear|
|2012-10-01 13:00:00|  Kansas City|       sky is

In [8]:
CLEAR_WEATHER_VALUES = ["sky is clear"]
SPRING_MONTHS = [3, 4, 5]
MIN_CLEAR_HOURS_PER_DAY = 18
MIN_CLEAR_DAYS_PER_MONTH = 15
filtered = (
        long_df
        .filter(F.col("weather_description").isNotNull())
        .filter(F.col("weather_description").isin(CLEAR_WEATHER_VALUES))
        .withColumn("date", F.to_date("timestamp"))
        .withColumn("year", F.year("timestamp"))
        .withColumn("month", F.month("timestamp"))
        .filter(F.col("month").isin(SPRING_MONTHS))
    )

filtered.show()

+-------------------+-----------+-------------------+----------+----+-----+
|          timestamp|       city|weather_description|      date|year|month|
+-------------------+-----------+-------------------+----------+----+-----+
|2013-03-01 00:00:00|Los Angeles|       sky is clear|2013-03-01|2013|    3|
|2013-03-01 00:00:00|San Antonio|       sky is clear|2013-03-01|2013|    3|
|2013-03-01 00:00:00|    Atlanta|       sky is clear|2013-03-01|2013|    3|
|2013-03-01 00:00:00|      Eilat|       sky is clear|2013-03-01|2013|    3|
|2013-03-01 01:00:00|Los Angeles|       sky is clear|2013-03-01|2013|    3|
|2013-03-01 01:00:00|  Las Vegas|       sky is clear|2013-03-01|2013|    3|
|2013-03-01 01:00:00|San Antonio|       sky is clear|2013-03-01|2013|    3|
|2013-03-01 01:00:00|  Charlotte|       sky is clear|2013-03-01|2013|    3|
|2013-03-01 01:00:00|      Eilat|       sky is clear|2013-03-01|2013|    3|
|2013-03-01 02:00:00|Los Angeles|       sky is clear|2013-03-01|2013|    3|
|2013-03-01 

In [9]:
daily_clear_hours = (
        filtered
        .groupBy("city", "year", "month", "date")
        .count()
        .withColumnRenamed("count", "clear_hours")
    )
daily_clear_hours.filter(F.col("city") == 'Saint Louis').orderBy("date").show()

+-----------+----+-----+----------+-----------+
|       city|year|month|      date|clear_hours|
+-----------+----+-----+----------+-----------+
|Saint Louis|2013|    3|2013-03-04|         13|
|Saint Louis|2013|    3|2013-03-05|          8|
|Saint Louis|2013|    3|2013-03-07|          5|
|Saint Louis|2013|    3|2013-03-08|         19|
|Saint Louis|2013|    3|2013-03-09|         17|
|Saint Louis|2013|    3|2013-03-10|          6|
|Saint Louis|2013|    3|2013-03-11|          2|
|Saint Louis|2013|    3|2013-03-13|          6|
|Saint Louis|2013|    3|2013-03-14|          7|
|Saint Louis|2013|    3|2013-03-15|          9|
|Saint Louis|2013|    3|2013-03-16|          6|
|Saint Louis|2013|    3|2013-03-17|          6|
|Saint Louis|2013|    3|2013-03-18|          2|
|Saint Louis|2013|    3|2013-03-19|          2|
|Saint Louis|2013|    3|2013-03-20|         12|
|Saint Louis|2013|    3|2013-03-21|         10|
|Saint Louis|2013|    3|2013-03-22|         11|
|Saint Louis|2013|    3|2013-03-23|     

In [10]:
clear_days = (
        daily_clear_hours
        .filter(F.col("clear_hours") >= MIN_CLEAR_HOURS_PER_DAY)
    )

In [11]:
monthly_clear_days = (
        clear_days
        .groupBy("city", "year", "month")
        .count()
        .withColumnRenamed("count", "clear_days")
        .filter(F.col("clear_days") >= MIN_CLEAR_DAYS_PER_MONTH)
    )

In [12]:
monthly_clear_days.show()

+-----------------+----+-----+----------+
|             city|year|month|clear_days|
+-----------------+----+-----+----------+
|      Kansas City|2014|    3|        18|
|        Beersheba|2017|    5|        22|
|            Eilat|2016|    3|        17|
|        Las Vegas|2013|    4|        23|
|            Eilat|2013|    3|        20|
|      Los Angeles|2014|    4|        17|
|            Eilat|2016|    4|        21|
|        Beersheba|2015|    5|        16|
|        Beersheba|2016|    4|        16|
|Tel Aviv District|2015|    5|        15|
|        Las Vegas|2017|    4|        23|
|            Haifa|2016|    4|        16|
|        Las Vegas|2014|    3|        30|
|        San Diego|2015|    3|        17|
|Tel Aviv District|2016|    4|        17|
|        Las Vegas|2013|    3|        26|
|        Las Vegas|2014|    4|        27|
|          Phoenix|2015|    4|        18|
|          Phoenix|2017|    4|        22|
|          Phoenix|2014|    5|        16|
+-----------------+----+-----+----

In [13]:
result = (
        monthly_clear_days
        .groupBy("city", "year")
        .agg(F.countDistinct("month").alias("valid_months"))
        .filter(F.col("valid_months") == len(SPRING_MONTHS))
        .select("year", "city")
        .orderBy("year", "city")
    )

result.show(truncate=False)

+----+---------+
|year|city     |
+----+---------+
|2013|Eilat    |
|2013|Las Vegas|
|2014|Las Vegas|
|2016|Eilat    |
|2016|Las Vegas|
|2016|Phoenix  |
|2017|Eilat    |
|2017|Las Vegas|
|2017|Phoenix  |
+----+---------+



In [14]:
from pyspark.sql.types import StructType, StructField, StringType
rows = []
for month in [3, 4, 5]:
    for day in range(1, 16):
        for hour in range(18):
            rows.append(
                (f"2021-{month:02d}-{day:02d} {hour:02d}:00:00", "sky is clear", "cloudy")
            )

print(rows)   
schema = StructType([
    StructField("datetime", StringType(), True),
    StructField("Rome", StringType(), True),
    StructField("Paris", StringType(), True),
    ])

nino = spark.createDataFrame(
        rows,
        schema=schema
    )
nino.show()

[('2021-03-01 00:00:00', 'sky is clear', 'cloudy'), ('2021-03-01 01:00:00', 'sky is clear', 'cloudy'), ('2021-03-01 02:00:00', 'sky is clear', 'cloudy'), ('2021-03-01 03:00:00', 'sky is clear', 'cloudy'), ('2021-03-01 04:00:00', 'sky is clear', 'cloudy'), ('2021-03-01 05:00:00', 'sky is clear', 'cloudy'), ('2021-03-01 06:00:00', 'sky is clear', 'cloudy'), ('2021-03-01 07:00:00', 'sky is clear', 'cloudy'), ('2021-03-01 08:00:00', 'sky is clear', 'cloudy'), ('2021-03-01 09:00:00', 'sky is clear', 'cloudy'), ('2021-03-01 10:00:00', 'sky is clear', 'cloudy'), ('2021-03-01 11:00:00', 'sky is clear', 'cloudy'), ('2021-03-01 12:00:00', 'sky is clear', 'cloudy'), ('2021-03-01 13:00:00', 'sky is clear', 'cloudy'), ('2021-03-01 14:00:00', 'sky is clear', 'cloudy'), ('2021-03-01 15:00:00', 'sky is clear', 'cloudy'), ('2021-03-01 16:00:00', 'sky is clear', 'cloudy'), ('2021-03-01 17:00:00', 'sky is clear', 'cloudy'), ('2021-03-02 00:00:00', 'sky is clear', 'cloudy'), ('2021-03-02 01:00:00', 'sky i

Traceback (most recent call last):
  File "c:\Users\g.aloisio\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyspark\serializers.py", line 437, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\g.aloisio\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyspark\cloudpickle\cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "c:\Users\g.aloisio\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyspark\cloudpickle\cloudpickle_fast.py", line 563, in dump
    return Pickler.dump(self, obj)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\g.aloisio\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyspark\cloudpickle\cloudpickle_fast.py", line 653, in reducer_override
    return self._function_reduce(obj)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\g.aloisio\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyspark\cloudpickle\cloudpickle_fast.py

PicklingError: Could not serialize object: IndexError: tuple index out of range

#### TASK 2

In [12]:
df = spark.read.csv("C:\\Users\\g.aloisio\\Desktop\\weather-spark-app\\data\\raw\\humidity.csv", header=True, inferSchema=True)

df.show()

+-------------------+---------+--------+-------------+-------+-----------+---------+---------+-------+-----------+------+-----------+------+-------+-----------+-----------+-----------+-------+---------+------------+-------+-------+------------+---------+-----+----------+-------+------------+--------+--------+------+---------+-----------------+-----+-----+---------+---------+
|           datetime|Vancouver|Portland|San Francisco|Seattle|Los Angeles|San Diego|Las Vegas|Phoenix|Albuquerque|Denver|San Antonio|Dallas|Houston|Kansas City|Minneapolis|Saint Louis|Chicago|Nashville|Indianapolis|Atlanta|Detroit|Jacksonville|Charlotte|Miami|Pittsburgh|Toronto|Philadelphia|New York|Montreal|Boston|Beersheba|Tel Aviv District|Eilat|Haifa|Nahariyya|Jerusalem|
+-------------------+---------+--------+-------------+-------+-----------+---------+---------+-------+-----------+------+-----------+------+-------+-----------+-----------+-----------+-------+---------+------------+-------+-------+------------+

In [3]:
def reshape_wide_to_long(
    df: DataFrame,
    datetime_col: str = "datetime",
    value_col_name: str = "value"
) -> DataFrame:
    """
    Convert a wide Spark DataFrame:
        datetime | city1 | city2 | city3 | ...

    Into a long format:
        datetime | city | value

    - Ignores the datetime column
    - Keeps null values (filtered later if needed)
    """

    city_columns = [c for c in df.columns if c != datetime_col]

    if not city_columns:
        raise ValueError("No city columns found to reshape")

    # build stack expression
    # stack(n, 'city1', city1, 'city2', city2, ...)
    stack_expr = ", ".join(
        [f"'{c}', `{c}`" for c in city_columns]
    )

    return (
        df.selectExpr(
            f"`{datetime_col}`",
            f"stack({len(city_columns)}, {stack_expr}) as (city, {value_col_name})"
        )
    )

In [4]:
humidity = spark.read.csv("C:\\Users\\g.aloisio\\Desktop\\weather-spark-app\\data\\raw\\humidity.csv", header=True, inferSchema=True)
temperature = spark.read.csv("C:\\Users\\g.aloisio\\Desktop\\weather-spark-app\\data\\raw\\temperature.csv", header=True, inferSchema=True)
pressure = spark.read.csv("C:\\Users\\g.aloisio\\Desktop\\weather-spark-app\\data\\raw\\pressure.csv", header=True, inferSchema=True)
humidity = reshape_wide_to_long(humidity)
temperature = reshape_wide_to_long(temperature)
pressure = reshape_wide_to_long(pressure)

In [5]:
pressure= pressure.withColumnRenamed("value", "pressure")
temperature= temperature.withColumnRenamed("value", "temperature")
humidity= humidity.withColumnRenamed("value", "humidity")

In [31]:
city_attributes = spark.read.csv("C:\\Users\\g.aloisio\\Desktop\\weather-spark-app\\data\\raw\\city_attributes.csv", header=True, inferSchema=True)
a = humidity.join(city_attributes, on="city", how="inner").filter(F.col("country") == "Israel")
a = a.withColumn("year", F.year("datetime")).withColumn("month", F.month("datetime"))

a.filter("year == 2017 and month == 9 and humidity is not null").show(200)

+-----------------+-------------------+--------+-------+---------+---------+----+-----+
|             city|           datetime|humidity|Country| Latitude|Longitude|year|month|
+-----------------+-------------------+--------+-------+---------+---------+----+-----+
|        Beersheba|2017-09-01 00:00:00|    90.0| Israel| 31.25181|34.791302|2017|    9|
|Tel Aviv District|2017-09-01 00:00:00|    61.0| Israel|32.083328|34.799999|2017|    9|
|            Eilat|2017-09-01 00:00:00|   100.0| Israel| 29.55805|34.948212|2017|    9|
|            Haifa|2017-09-01 00:00:00|   100.0| Israel|32.815559| 34.98917|2017|    9|
|        Nahariyya|2017-09-01 00:00:00|   100.0| Israel|33.005859| 35.09409|2017|    9|
|        Jerusalem|2017-09-01 00:00:00|    61.0| Israel|31.769039|35.216331|2017|    9|
|        Beersheba|2017-09-01 01:00:00|    90.0| Israel| 31.25181|34.791302|2017|    9|
|Tel Aviv District|2017-09-01 01:00:00|    64.0| Israel|32.083328|34.799999|2017|    9|
|            Eilat|2017-09-01 01

In [5]:
from pyspark.sql import DataFrame
from functools import reduce


def join_metrics(
    dfs: list[DataFrame],
    join_cols: list[str] = ["datetime", "city"]
) -> DataFrame:
    """
    Join multiple metric DataFrames on (datetime, city) using FULL OUTER JOIN.

    Expected schema for each df:
        datetime | city | <metric>

    Returns:
        datetime | city | metric1 | metric2 | ...
    """

    if not dfs:
        raise ValueError("No DataFrames provided for join")

    def _join(left: DataFrame, right: DataFrame) -> DataFrame:
        return left.join(right, on=join_cols, how="full")

    return reduce(_join, dfs)


In [11]:
metrics_df = join_metrics([
    temperature,
    pressure,
    humidity
])


In [12]:
metrics_df.show()

+-------------------+-----------------+-------------+--------+--------+
|           datetime|             city|  temperature|pressure|humidity|
+-------------------+-----------------+-------------+--------+--------+
|2012-10-01 12:00:00|     Indianapolis|         null|    null|    null|
|2012-10-01 12:00:00|        Las Vegas|         null|    null|    null|
|2012-10-01 12:00:00|       Pittsburgh|         null|    null|    null|
|2012-10-01 12:00:00|Tel Aviv District|         null|    null|    null|
|2012-10-01 12:00:00|          Toronto|         null|    null|    null|
|2012-10-01 12:00:00|        Vancouver|         null|    null|    null|
|2012-10-01 13:00:00|          Chicago|       284.01|  1014.0|    71.0|
|2012-10-01 13:00:00|            Haifa|        304.4|  1013.0|    51.0|
|2012-10-01 13:00:00|     Indianapolis|       283.85|  1011.0|    76.0|
|2012-10-01 13:00:00|     Jacksonville|       298.17|  1009.0|    88.0|
|2012-10-01 13:00:00|        Jerusalem|        303.5|   990.0|  

In [13]:
CITY_TIMEZONE = {
    "Phoenix": "America/Phoenix",
    "Nahariyya": "Asia/Jerusalem",
    "Dallas": "America/Chicago",
    "San Antonio": "America/Chicago",
    "Philadelphia": "America/New_York",
    "Los Angeles": "America/Los_Angeles",
    "Indianapolis": "America/Indiana/Indianapolis",
    "San Francisco": "America/Los_Angeles",
    "San Diego": "America/Los_Angeles",
    "Nashville": "America/Chicago",
    "Detroit": "America/Detroit",
    "Portland": "America/Los_Angeles",
    "Haifa": "Asia/Jerusalem",
    "Montreal": "America/Montreal",
    "Jerusalem": "Asia/Jerusalem",
    "Pittsburgh": "America/New_York",
    "Beersheba": "Asia/Jerusalem",
    "Chicago": "America/Chicago",
    "Toronto": "America/Toronto",
    "Vancouver": "America/Vancouver",
    "Tel Aviv District": "Asia/Jerusalem",
    "Atlanta": "America/New_York",
    "Las Vegas": "America/Los_Angeles",
    "Seattle": "America/Los_Angeles",
    "Kansas City": "America/Chicago",
    "Saint Louis": "America/Chicago",
    "Minneapolis": "America/Chicago",
    "Houston": "America/Chicago",
    "Jacksonville": "America/New_York",
    "Albuquerque": "America/Denver",
    "Miami": "America/New_York",
    "New York": "America/New_York",
    "Charlotte": "America/New_York",
    "Eilat": "Asia/Jerusalem",
    "Denver": "America/Denver",
    "Boston": "America/New_York"
}

from pyspark.sql import DataFrame
from pyspark.sql import functions as F


def convert_datetime_to_local(df: DataFrame, datetime_col: str = "datetime") -> DataFrame:
    """
    Add local datetime, year, month columns based on city timezone mapping.
    """
    # aggiunge la colonna timezone
    df = df.withColumn(
        "timezone",
        F.create_map(
            *[F.lit(x) for x in sum(CITY_TIMEZONE.items(), ())]
        )[F.col("city")]
    )

    # converte UTC → locale
    df = df.withColumn("datetime_local", F.from_utc_timestamp(F.col(datetime_col), F.col("timezone")))

    # estrae year e month
    df = df.withColumn("year", F.year("datetime_local"))
    df = df.withColumn("month", F.month("datetime_local"))

    return df


In [14]:
metrics_df = convert_datetime_to_local(metrics_df)

In [16]:
from pyspark.sql import DataFrame
from pyspark.sql import functions as F


def aggregate_nation_stats(metrics_df: DataFrame, city_attr_df: DataFrame) -> DataFrame:
    """
    Aggrega temperature, pressure, humidity per nazione e mese/anno.

    Args:
        metrics_df: df con colonne ['city', 'datetime_local', 'year', 'month', 'temperature', 'pressure', 'humidity']
        city_attr_df: df con colonne ['city', 'country']

    Returns:
        DataFrame con schema:
        ['country', 'year', 'month',
         'avg_temperature', 'std_temperature', 'min_temperature', 'max_temperature',
         'avg_pressure', 'std_pressure', 'min_pressure', 'max_pressure',
         'avg_humidity', 'std_humidity', 'min_humidity', 'max_humidity']
    """

    # Join città → nazione
    df = metrics_df.join(city_attr_df.select("city", "country"), on="city", how="left")

    # Raggruppa per nazione + anno + mese
    agg_df = df.groupBy("country", "year", "month").agg(
        F.avg("temperature").alias("avg_temperature"),
        F.stddev("temperature").alias("std_temperature"),
        F.min("temperature").alias("min_temperature"),
        F.max("temperature").alias("max_temperature"),

        F.avg("pressure").alias("avg_pressure"),
        F.stddev("pressure").alias("std_pressure"),
        F.min("pressure").alias("min_pressure"),
        F.max("pressure").alias("max_pressure"),

        F.avg("humidity").alias("avg_humidity"),
        F.stddev("humidity").alias("std_humidity"),
        F.min("humidity").alias("min_humidity"),
        F.max("humidity").alias("max_humidity")
    )

    return agg_df


In [17]:
city_attributes = spark.read.csv("C:\\Users\\g.aloisio\\Desktop\\weather-spark-app\\data\\raw\\city_attributes.csv", header=True, inferSchema=True)
nation_stats_df = aggregate_nation_stats(metrics_df, city_attributes)
nation_stats_df.show()

+-------------+----+-----+------------------+------------------+---------------+---------------+------------------+------------------+------------+------------+-----------------+------------------+------------+------------+
|      country|year|month|   avg_temperature|   std_temperature|min_temperature|max_temperature|      avg_pressure|      std_pressure|min_pressure|max_pressure|     avg_humidity|      std_humidity|min_humidity|max_humidity|
+-------------+----+-----+------------------+------------------+---------------+---------------+------------------+------------------+------------+------------+-----------------+------------------+------------+------------+
|       Israel|2014|    2|286.65789560500673| 4.718883014228063|  274.754820495|         304.69|1022.1031289111389|11.013097211385835|       957.0|      1039.0|81.26695563488016|20.108666137405198|        10.0|       100.0|
|United States|2015|   10|  289.643314356404| 6.842906909831214|        269.246|         313.16|1019.296

In [20]:
nation_stats_df.orderBy("country", "year", "month").show(100)

+-------+----+-----+------------------+------------------+---------------+---------------+------------------+------------------+------------+------------+------------------+------------------+------------+------------+
|country|year|month|   avg_temperature|   std_temperature|min_temperature|max_temperature|      avg_pressure|      std_pressure|min_pressure|max_pressure|      avg_humidity|      std_humidity|min_humidity|max_humidity|
+-------+----+-----+------------------+------------------+---------------+---------------+------------------+------------------+------------+------------+------------------+------------------+------------+------------+
| Canada|2012|   10| 283.4482825646639|3.7536694858351236|         271.73|          293.9|1009.3456352022711|15.350050795196864|       807.0|      1032.0| 72.70319084888621|19.899698385564697|         7.0|       100.0|
| Canada|2012|   11|277.48219602404066|4.6018384374416765|         262.87|         290.63|1016.4773801485483|10.151271130463

### Task 3

In [ ]:

temperature = spark.read.csv("C:\\Users\\g.aloisio\\Desktop\\weather-spark-app\\data\\raw\\temperature.csv", header=True, inferSchema=True)
city_attributes = spark.read.csv("C:\\Users\\g.aloisio\\Desktop\\weather-spark-app\\data\\raw\\city_attributes.csv", header=True, inferSchema=True)

In [2]:
from pyspark.sql import DataFrame
from pyspark.sql import functions as F


def add_season(df: DataFrame) -> DataFrame:
    """
    Aggiunge la colonna 'season':
    - cold: gennaio-aprile
    - hot: giugno-settembre
    Esclude gli altri mesi.
    """
    return (
        df
        .withColumn(
            "season",
            F.when(F.col("month").between(1, 4), F.lit("cold"))
             .when(F.col("month").between(6, 9), F.lit("hot"))
        )
        .filter(F.col("season").isNotNull())
    )


In [3]:
def test_add_season_assignment(spark):
    rows = [
        ("Rome", 1),   # Jan
        ("Rome", 4),   # Apr
        ("Rome", 6),   # Jun
        ("Rome", 9),   # Sep
        ("Rome", 12),  # Dec (ignored)
    ]

    df = spark.createDataFrame(rows, ["city", "month"])

    result = add_season(df).select("month", "season").collect()

    mapping = {r["month"]: r["season"] for r in result}

    assert mapping[1] == "cold"
    assert mapping[4] == "cold"
    assert mapping[6] == "hot"
    assert mapping[9] == "hot"
    assert mapping[12] is None

In [4]:
test_add_season_assignment(spark)

NameError: name 'spark' is not defined